<a href="https://colab.research.google.com/github/tunakan12/saikikannsu/blob/main/cDA25_11_%E6%BC%94%E7%BF%92%E8%AA%B2%E9%A1%8C_%E3%82%BD%E3%83%BC%E3%83%882_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#（１）共通ツール：データ生成，配列の出力などの関数群

##(1a)ヘッダーファイル

構造体：counter
  - compare: 比較回数
  - swap: 交換回数
  - move: 移動回数

準備した関数
- 基本関数
  - swap：指定した２データの交換
  - copyarray：配列データの複写
  - printarray：配列データの出力
- データ読み込み関数
  - allocate_member：新しいメンバを割り当てる
  - read_members_from_file：ファイルからメンバを読み込む
  - init_members_from_file：新しいメンバを読み込む
  - add_members_from_file：追加のメンバを読み込む
- 比較関数
  - compare_by_yomi：名前(読み)(yomi)で比較
  - compare_by_age：年齢(age)で比較
  - compare_by_height：身長(height)で比較
  - compare_by_score：強さ(score)で比較
  - compare_by_group：所属(group)で比較
- ソート関数
  - bubble_sort：バブルソート
  - bubble2_sort：改良バブルソート
  - shaker_sort：シェーカーソート
  - select_sort：選択ソート
  - insert_sort：挿入ソート
  - quick_sort：クイックソート
  - merge_sort：マージソート
  - heap_sort：ヒープソート

In [54]:
%%file main.h
#define MAX_NAME  (12*3)
#define MAX_ATTR  (3*3)
// Member構造体の定義
typedef struct member {
    char  name[MAX_NAME+1];  // 名前(漢字表記)
    char  yomi[MAX_NAME+1];  // 読み(カタカナ表記)
    char  group[MAX_NAME+1]; // 所属
    char  attr[MAX_ATTR+1];  // 属性
    int   age;               // 年齢
    float height;            // 身長
    int   score;             // 強さ
} Member;
// 比較関数型へのポインタ
typedef int (*CmpFunc)(const Member *, const Member *, int);
// カウンタ型
typedef struct counter{
    int compare;
    int swap;
    int move;
} Counter;

// tool functions
Member *allocate_member(const char *name, const char *yomi, const char *group, const char *attr, int age, float height, int score);
Member **init_members_from_file(const char *filename, int *ret_num);
Member **add_members_from_file(const char *filename, int *ret_num, Member **member_array);
void swap(Member *data[], int idx1, int idx2);
void copyarray(int n, Member *data1[], Member *data2[]);
void printarray(int n, Member *data[]);

// counter functions
extern Counter ctr;
void print_counter(void);
void reset_counter(void);

// compare functions
int compare_by_yomi(const Member *data1, const Member *data2, int descending);
int compare_by_age(const Member *data1, const Member *data2, int descending);
int compare_by_height(const Member *data1, const Member *data2, int descending);
int compare_by_score(const Member *data1, const Member *data2, int descending);
int compare_by_group(const Member *data1, const Member *data2, int descending);

// sort functions
void launch_sort(int n, Member *data[], void (*sort_func)(int, Member**, CmpFunc, int), CmpFunc cmp, int descending, const char *sort_name, const char *cmp_name);
#define LAUNCH_SORT(n, data, sort_func, cmp, descending)  launch_sort(n, data, sort_func, cmp, descending, #sort_func, #cmp)
void bubble_sort(int n, Member *data[], CmpFunc cmp, int descending);
void bubble2_sort(int n, Member *data[], CmpFunc cmp, int descending);
void shaker_sort(int n, Member *data[], CmpFunc cmp, int descending);
void select_sort(int n, Member *data[], CmpFunc cmp, int descending);
void insert_sort(int n, Member *data[], CmpFunc cmp, int descending);
void quick_sort(int n, Member *data[], CmpFunc cmp, int descending);
void merge_sort(int n, Member *data[], CmpFunc cmp, int descending);
void heap_sort(int n, Member *data[], CmpFunc Cmp, int descending);
Member **merge_two_arrays(int n1, Member *data1[], int n2, Member *data2[], CmpFunc cmp, int descending, int *num_merged);
Member **bucket_sort(int max_index, int n, Member *data[], int (*get_index)(const Member*), int descending);
Member **merge_three_arrays(int n1, Member *data1[], int n2, Member *data2[], int n3, Member *data3[], CmpFunc cmp, int descending, int *num_merged);

Overwriting main.h


##(1b)データ生成，配列の出力，コピー

基本関数
- swap
 - 処理：指定した２データの交換
 - 入力：data[](Member*)：配列データ
 - 入力：idx1, 1dx2(int)：交換データのインデックス
- copyarray
 - 処理：配列データの複写
 - 入力：n(int)：データサイズ，data1[](Member*)：入力元配列データ
 - 出力：data2[](int)：出力先配列データ
- printarray
 - 処理：配列データの出力
 - 入力：n(int)：データサイズ，data[](Member*)：配列データ
 - 出力：出力先への配列データの出力

カウンタ操作
- print_counter
 - 処理：カウンタを表示(lap, all)
- reset_counter
 - 処理：現在のカウンタをトータルカウンタに足しこみ、リセット

メンバの割り当てと読み込み
- allocate_member
 - 処理：新しいメンバ用の構造体を確保し代入
 - 入力：メンバの情報
 - 出力：Member型へのポインタ
- read_members_from_file
 - 処理：CSVファイルからのメンバ情報の読み出し
 - 入力：filename(char\*):ファイル名，ret_num(int\*):データサイズへのポインタ，member_array(Member\*\*):メンバへの配列
 - 出力：情報格納後のメンバへの配列
- init_members_from_file
 - 処理：CSVファイルからの初期メンバの読み出し
- add_members_from_file
 - 処理：CSVファイルからの追加メンバの読み出し

ソート関数の呼び出し
- launch_sort
 - 処理：ソート関数の呼び出しを行う。LAUNCH_SORTマクロを使うことにより、ソート関数名、比較関数名、昇順/降順を自動的に表示する。

In [55]:
%%file tool.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include "main.h"
void swap(Member *data[], int idx1, int idx2) {
    Member *temp;
    if (idx1 != idx2) {
        temp = data[idx1];
        data[idx1] = data[idx2];
        data[idx2] = temp;
    }
    return;
}
void printarray(int n, Member *data[]) {
    for(int i=0; i<n; i++){
        printf("%s(%s): 年齢:%d, 身長:%3.1lf, 強さ:%d, 所属:%s, 属性:%s\n",
               data[i]->name, data[i]->yomi, data[i]->age, data[i]->height,
               data[i]->score, data[i]->group, data[i]->attr);
    }
    return;
}
void copyarray(int n, Member *data1[], Member *data2[]) {
    for(int i=0; i<n; i++){
        data2[i] = data1[i];
    }
    return;
}
/**
 * カウンタの操作
 */
Counter ctr;
Counter total_ctr;
void print_counter(void) {
    printf("lap: compare = %d, swap = %d, move = %d\n", ctr.compare, ctr.swap, ctr.move);
    printf("all: compare = %d, swap = %d, move = %d\n",
           total_ctr.compare+ctr.compare, total_ctr.swap+ctr.swap, total_ctr.move+ctr.move);
}
void reset_counter(void) {
    total_ctr.compare += ctr.compare;
    total_ctr.swap    += ctr.swap;
    total_ctr.move    += ctr.move;
    ctr.compare = ctr.swap = ctr.move = 0;
}

/**
 * メンバの割り当てと読み込み
 */
Member *allocate_member(const char *name, const char *yomi, const char *group, const char *attr, int age, float height, int score) {
    Member *new_member;
    new_member = (Member *)malloc(sizeof(Member));
    if (new_member == NULL) {
        printf("Cannot allocate new member\n");
        return NULL;
    }
    strncpy(new_member->name, name, MAX_NAME);
    strncpy(new_member->yomi, yomi, MAX_NAME);
    strncpy(new_member->group, group, MAX_NAME);
    strncpy(new_member->attr, attr, MAX_ATTR);
    new_member->age = age;
    new_member->height = height;
    new_member->score = score;
    return new_member;
}
Member **read_members_from_file(const char *filename, int *ret_num, Member **member_array) {
    FILE *fin;
    char buf[BUFSIZ];
    int num_members, i;
    char name[MAX_NAME+1];
    char yomi[MAX_NAME+1];
    char group[MAX_NAME+1];
    char attr[MAX_ATTR+1];
    int age;
    float height;
    int score;

    fin = fopen(filename, "r");
    if (fin == NULL) {
        printf("Cannot open %s\n", filename);
        return NULL;
    }
    if (fgets(buf, BUFSIZ, fin) == NULL || sscanf(buf, "%d", &num_members) < 1) {
        printf("Illegal format %s\n", filename);
        fclose(fin);
        return NULL;
    }
    if (member_array == NULL) {
        member_array = (Member**)malloc(sizeof(Member) * num_members);
        if (member_array == NULL) {
            printf("Cannot allocate array\n");
            fclose(fin);
            return NULL;
        }
    } else {
        Member **tmp_member_array = (Member**)realloc(member_array, (sizeof(Member) * (num_members + *ret_num)));
        if (tmp_member_array == NULL) {
            printf("Cannot reallocate array\n");
            fclose(fin);
            return member_array;
        }
        member_array = tmp_member_array;
    }
    for (i = 0; i < num_members; i++) {
        if (fgets(buf, BUFSIZ, fin) == NULL ||
            sscanf(buf, "%[^,],%[^,],%[^,],%[^,],%d,%f,%d",
                   name, yomi, group, attr, &age, &height, &score) < 7){
            printf("Illegal format %s in line %d\n", filename, i+1);
            break;
        }
        member_array[*ret_num + i] = allocate_member(name, yomi, group ,attr, age, height, score);
    }
    fclose(fin);
    *ret_num += i;
    return member_array;
}
Member **init_members_from_file(const char *filename, int *ret_num) {
    *ret_num = 0;
    return read_members_from_file(filename, ret_num, NULL);
}
Member **add_members_from_file(const char *filename, int *ret_num, Member **member_array) {
    return read_members_from_file(filename, ret_num, member_array);
}
/**
 * ソート関数を呼び出し、その時のソート関数名、比較関数名、昇順か降順かを表示する
 */
void launch_sort(int n, Member *data[], void (*sort_func)(int, Member**, CmpFunc, int), CmpFunc cmp, int descending,
                 const char *sort_name, const char *cmp_name) {
    printf("\n%s with %s(%s)\n", sort_name, cmp_name, descending ? "descending" : "ascending");
    sort_func(n, data, cmp, descending);
}

Overwriting tool.c


##(1c)比較関数

In [56]:
%%file compare.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"

// 名前(読み)(yomi)で比較
int compare_by_yomi(const Member *data1, const Member *data2, int descending) {
    int result = strcmp(data1->yomi, data2->yomi);
    if (descending) {
        result = -result;
    }
    return result;
}
// 年齢(age)で比較
int compare_by_age(const Member *data1, const Member *data2, int descending) {
    int result;
    if (data1->age > data2->age) {
        result = 1;
    } else if (data1->age < data2->age) {
        result = -1;
    } else {
        result = 0;
    }
    if (descending) {
        result = -result;
    }
    return result;
}
// 身長(height)で比較
int compare_by_height(const Member *data1, const Member *data2, int descending) {
    int result;
    if (data1->height > data2->height) {
        result = 1;
    } else if (data1->height < data2->height) {
        result = -1;
    } else {
        result = 0;
    }
    if (descending) {
        result = -result;
    }
    return result;
}
// 強さ(score)で比較
int compare_by_score(const Member *data1, const Member *data2, int descending) {
    int result;
    if (data1->score > data2->score) {
        result = 1;
    } else if (data1->score < data2->score) {
        result = -1;
    } else {
        result = 0;
    }
    if (descending) {
        result = -result;
    }
    return result;
}
// 所属(group)で比較
int compare_by_group(const Member *data1, const Member *data2, int descending) {
    int result = strcmp(data1->group, data2->group);
    if (descending) {
        result = -result;
    }
    return result;
}

Overwriting compare.c


##(1d)メンバ定義

最初の1行目に全部で何人分のデータが続くかを示す整数。
以下、各行に

名前,読み,所属,属性,年齢,身長,強さ

の順に与える。

In [57]:
%%file members.csv
15
竈門炭治郎,カマドタンジロウ,蒲鉾,水,16,165.0,40
冨岡義勇,トミオカギユウ,柱,水,21,176.0,80
竈門禰豆子,カマドネズコ,蒲鉾,鬼,14,150.0,30
我妻善逸,アガツマゼンイツ,蒲鉾,雷,16,164.5,40
栗花落カナヲ,ツユリカナヲ,五感,花,16,156.0,50
嘴平伊之助,ハシビライノスケ,蒲鉾,獣,15,164.0,40
胡蝶しのぶ,コチョウシノブ,柱,蟲,18,151.0,70
煉獄杏寿郎,レンゴクキョウジュロウ,柱,炎,20,177.0,80
宇随天元,ウズイテンゲン,柱,音,23,198.0,80
時透無一郎,トキトウムイチロウ,柱,霞,14,160.0,90
甘露寺蜜璃,カンロジミツリ,柱,恋,19,167.0,70
伊黒小芭内,イグロオバナイ,柱,蛇,21,162.0,70
不死川玄弥,シナズガワゲンヤ,五感,鬼,16,180.0,30
不死川実弥,シナズガワサネミ,柱,風,21,179.0,90
悲鳴嶋行冥,ヒメジマギョウメイ,柱,岩,27,220.0,100


Overwriting members.csv


In [58]:
%%file members3.csv
16
鬼舞辻無惨,キブツジムザン,始祖,鬼,1100,185.0,1000
塁,ルイ,下弦,鬼,150,148.0,50
轆轤,ロクロ,下弦,鬼,200,172.5,45
病葉,ワクラバ,下弦,鬼,180,165.0,40
零余子,ムカゴ,下弦,鬼,160,152.5,35
釜鵺,カマヌエ,下弦,鬼,190,168.5,30
猗窩座,アカザ,上弦,鬼,600,172.5,300
魘夢,エンム,下弦,鬼,150,168.0,80
童磨,ドウマ,上弦,鬼,300,174.4,400
妓夫太郎,ギュウタロウ,上弦,鬼,200,173.5,180
堕姫,ダキ,上弦,鬼,192,155.5,160
半天狗,ハンテング,上弦,鬼,400,169.5,250
玉壺,ギョッコ,上弦,鬼,300,155.0,200
黒死牟,コクシボウ,上弦,鬼,1000,180.0,500
鳴女,ナキメ,上弦,鬼,200,155.0,30
獪岳,カイガク,上弦,鬼,30,167.0,160

Overwriting members3.csv


#（２）ソートプログラム群

- ソートプログラム群の共通形式
 - 入力 n(int)：データサイズ，data\[\](Member\*)：配列データ，cmp：比較関数，descending：0なら昇順・0以外なら降順
 - 出力 data\[\](Member\*)：ソートされた配列データ


##(2a)バブルソート オリジナル

In [59]:
%%file bubble_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
void bubble_sort(int n, Member *data[], CmpFunc cmp, int descending){
    reset_counter();
    for (int i=n-1; i>=1; i--) {
        for (int j=1; j<=i; j++) {
            ctr.compare++;
            if (cmp(data[j-1], data[j], descending)> 0) {
                swap(data, j-1, j);
                ctr.swap++;
            }
        }
    }
    return;
}

Overwriting bubble_sort.c


##(2b)バブルソート改良版

In [60]:
%%file bubble2_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
void bubble2_sort(int n, Member *data[], CmpFunc cmp, int descending){
    int jswap, jmax;
    reset_counter();
    jmax = n-1;
    for (int i=n-1; i>=1; i--) {
            jswap = 1;
        for (int j=1; j<=jmax; j++) {
            ctr.compare++;
            if (cmp(data[j-1], data[j], descending) > 0) {
                jswap = j;
                swap(data, j-1, j);
                ctr.swap++;
            }
        }
        jmax = jswap-1;
    }
    return;
}

Overwriting bubble2_sort.c


##(2c)シェーカーソート

In [61]:
%%file shaker_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
void shaker_sort(int n, Member *data[], CmpFunc cmp, int descending){
    int jmin=0, jmax=n-1, j, temp, jswap;
    reset_counter();
    while(1){
        if (jmin>=jmax) break;
        jswap = jmax;
        for (j=jmax; j>jmin; j--) {
            ctr.compare++;
            if (cmp(data[j-1], data[j], descending) > 0) {
                jswap = j;
                swap(data, j-1, j);
                ctr.swap++;
            }
        }
        jmin = jswap;
        if (jmin>=jmax) break;
        jswap = 0;
        for (j=jmin; j<jmax; j++) {
            ctr.compare++;
            if (cmp(data[j], data[j+1], descending) > 0) {
                jswap = j;
                swap(data, j, j+1);
                ctr.swap++;
            }
        }
        jmax = jswap;
    }
}

Overwriting shaker_sort.c


##(2d)選択ソート

In [62]:
%%file select_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
void select_sort(int n, Member *data[], CmpFunc cmp, int descending){
    int i_min;
    Member *min;
    reset_counter();
    for (int i=0; i<n-1; i++) {
        i_min=i;
        min=data[i];
        for (int j=i+1; j<n; j++) {
            if (cmp(min, data[j], descending) > 0){
                i_min=j;
                min=data[j];
            }
            ctr.compare++;
        }
        swap(data, i_min, i);
        ctr.swap++;
    }
    return;
}

Overwriting select_sort.c


##(2e)挿入ソート

In [63]:
%%file insert_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
void insert_sort(int n, Member *data[], CmpFunc cmp, int descending){
    reset_counter();
    for (int i=1; i<n; i++) {
        for (int j=i; j>0; j--) {
            ctr.compare++;
            if (cmp(data[j-1], data[j], descending) > 0) {
                swap(data, j-1, j);
                ctr.swap++;
            }
            else break;
        }
    }
    return;
}

Overwriting insert_sort.c


##(2f)クイックソート

In [64]:
%%file quick_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
void quicksort(Member *data[], CmpFunc cmp, int descending, int l, int r){
    int mid, i, j;
    Member *pivot;
    mid=(l+r)/2; pivot=data[mid];
    i=l; j=r;
    while (i<=j) {
        ctr.compare++;
        while (cmp(data[i], pivot, descending) < 0) {
            i=i+1;
            ctr.compare++;
        }
        ctr.compare++;
        while (cmp(pivot, data[j], descending) < 0) {
            j=j-1;
            ctr.compare++;
        }
        if(i<=j){
            swap(data, i, j);
            ctr.swap++;
            i=i+1; j=j-1;
        }
    }
    if (l<j)quicksort(data, cmp, descending, l, j);
    if (i<r)quicksort(data, cmp, descending, i, r);
    return;
}
void quick_sort(int n, Member *data[], CmpFunc cmp, int descending){
    reset_counter();
    quicksort(data, cmp, descending, 0, n-1);
    return;
}

Overwriting quick_sort.c


##(2g)ヒープソート

In [65]:
%%file heap_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
void downheap(int size, Member *data[], CmpFunc cmp, int descending, int n, int pt){
    int lt, rt;
    while(1){
        lt=2*pt+1;
        rt=2*pt+2;
        if(lt>n){
            return;
        }
        if(lt==n){
            ctr.compare++;
            if (lt < size && pt < size && cmp(data[lt], data[pt], descending) > 0) {
                swap(data, lt, pt);
                ctr.swap++;
            }
            return;
        }
        //(lt<n)
        ctr.compare++;
        if(lt < size && rt < size && cmp(data[lt], data[rt], descending) > 0){
            ctr.compare++;
            if (lt < size && pt < size && cmp(data[lt], data[pt], descending) > 0) {
                swap(data, lt, pt);
                ctr.swap++;
            }
            pt=lt;
        }
        else{
            ctr.compare++;
            if (rt < size && pt < size && cmp(data[rt], data[pt], descending) > 0) {
                swap(data, rt, pt);
                ctr.swap++;
            }
            pt=rt;
        }
    }
}
void heap_sort(int n, Member *data[], CmpFunc cmp, int descending){
    int m;
    reset_counter();
    m=n;
    if (m%2==0){m--;}
    m = (m-1)/2;
    for (int pt=m; pt>=0; pt--) {
        downheap(n, data, cmp, descending, n, pt);
    }
    for (int pt=n-1; pt>0; pt--){
        ctr.compare++;
        if (cmp(data[0], data[pt], descending) > 0) {
            swap(data, 0, pt);
            ctr.swap++;
        }
        downheap(n, data, cmp, descending, pt-1, 0);
    }
    return;
}

Overwriting heap_sort.c


##(2h)マージソート

In [66]:
%%file merge_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
void mergesort(Member *data[], Member *temp[], CmpFunc cmp, int descending, int lt, int rt) {
    int mid, i, j, k;
    mid = (lt+rt)/2;
    if((rt-lt)>1) {
        mergesort(data, temp, cmp, descending, lt, mid);
        mergesort(data, temp, cmp, descending, mid+1, rt);
    }
    for (i=lt; i<=mid; i++) {
        temp[i] = data[i];
        ctr.move++;
    }
    for (j=mid+1; j<=rt; j++) {
        temp[rt-(j-(mid+1))] = data[j];
        ctr.move++;
    }
    i=lt; j=rt;
    for (k=lt; k<=rt; k++) {
        if (cmp(temp[i], temp[j], descending) < 0){
            data[k] = temp[i++];
            ctr.move++;
        }
        else {
            data[k] = temp[j--];
            ctr.move++;
        }
        ctr.compare++;
    }
}
void merge_sort(int n, Member *data[], CmpFunc cmp, int descending){
    Member **temp = (Member**)malloc(n * sizeof(Member*));
    reset_counter();
    mergesort(data, temp, cmp, descending, 0, n-1);
    free(temp);
    return;
}

Overwriting merge_sort.c


##(2i)コンパイル

In [67]:
!gcc tool.c -c
!gcc compare.c -c
!gcc bubble_sort.c -c
!gcc bubble2_sort.c -c
!gcc shaker_sort.c -c
!gcc select_sort.c -c
!gcc insert_sort.c -c
!gcc quick_sort.c -c
!gcc merge_sort.c -c
!gcc heap_sort.c -c

#演習テスト 課題１ マージソート

複数のファイルからメンバをそれぞれ読み込み、それぞれをソートする。その上で2つの配列をマージするマージソートのプログラムを完成させよ。

##2本の配列をマージするマージソート

In [96]:
%%file merge_two_arrays.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
Member **merge_two_arrays(int num1, Member *data1[], int num2, Member *data2[],
                          CmpFunc cmp, int descending, int *num_merged){
    Member **merged = (Member**)malloc((num1+num2) * sizeof(Member*));
    int i, j, p;
    printf("merge_two_arrays.\n");
    reset_counter();
    i = 0; j = 0; p = 0;
    // ここから
    while (i < num1 || j < num2) {
        if (i == num1) {
            // 無条件に data2[] から入れる
            merged[p++] = data2[j++];

        } else if (j == num2) {
            // 無条件に data1[] から入れる
             merged[p++] = data1[i++];

        } else {
            // data1[i] と data2[j] を比較して先に入れる方を入れる
            int result = cmp(data1[i], data2[j], descending);
            if (result > 0) {
                // data2[] から入れる
                merged[p++] = data2[j++];
                j++;
            } else {
                // data1[] から入れる
                merged[p++] = data1[i++];
                i++;
            }
            ctr.compare++;
        }
        ctr.move++;
    }
    // ここまでを埋める
    *num_merged = num1 + num2;
    free(data1);
    free(data2);
    return merged;
}

Overwriting merge_two_arrays.c


###メインプログラム

In [97]:
%%file test1.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"

int main(void) {
    Member **member_array_1, **member_array_2, **merged_array;
    int num_members_1, num_members_2, num_merged;

    // メンバー1の読み込み
    member_array_1 = init_members_from_file("members.csv", &num_members_1);

    printf("initial 1:\n");
    printarray(num_members_1, member_array_1);

    // メンバー2の読み込み
    member_array_2 = init_members_from_file("members3.csv", &num_members_2);

    printf("initial 2:\n");
    printarray(num_members_2, member_array_2);

    // 名前の読み(yomi)順にソート
    LAUNCH_SORT(num_members_1, member_array_1, quick_sort, compare_by_yomi, 0);
    printarray(num_members_1, member_array_1);
    print_counter();

    // 名前の読み(yomi)順にソート
    LAUNCH_SORT(num_members_2, member_array_2, quick_sort, compare_by_yomi, 0);
    printarray(num_members_2, member_array_2);
    print_counter();

    // 2つの配列をマージ
    printf("\n");
    merged_array = merge_two_arrays(num_members_1, member_array_1, num_members_2, member_array_2,
                                    compare_by_yomi, 0, &num_merged);
    printf("num_merged = %d\n", num_merged);
    printarray(num_merged, merged_array);
    print_counter();

    LAUNCH_SORT(num_merged, merged_array, merge_sort, compare_by_yomi, 0);
    printarray(num_merged, merged_array);
    print_counter();

    return 0;
}

Overwriting test1.c


###コンパイル（test1.c）＆リンク

In [98]:
!gcc merge_two_arrays.c -c
!gcc test1.c -c
!gcc tool.o compare.o bubble_sort.o bubble2_sort.o shaker_sort.o insert_sort.o select_sort.o quick_sort.o heap_sort.o merge_sort.o merge_two_arrays.o test1.o -o test1

###実行

In [99]:
!./test1

initial 1:
竈門炭治郎(カマドタンジロウ): 年齢:16, 身長:165.0, 強さ:40, 所属:蒲鉾, 属性:水
冨岡義勇(トミオカギユウ): 年齢:21, 身長:176.0, 強さ:80, 所属:柱, 属性:水
竈門禰豆子(カマドネズコ): 年齢:14, 身長:150.0, 強さ:30, 所属:蒲鉾, 属性:鬼
我妻善逸(アガツマゼンイツ): 年齢:16, 身長:164.5, 強さ:40, 所属:蒲鉾, 属性:雷
栗花落カナヲ(ツユリカナヲ): 年齢:16, 身長:156.0, 強さ:50, 所属:五感, 属性:花
嘴平伊之助(ハシビライノスケ): 年齢:15, 身長:164.0, 強さ:40, 所属:蒲鉾, 属性:獣
胡蝶しのぶ(コチョウシノブ): 年齢:18, 身長:151.0, 強さ:70, 所属:柱, 属性:蟲
煉獄杏寿郎(レンゴクキョウジュロウ): 年齢:20, 身長:177.0, 強さ:80, 所属:柱, 属性:炎
宇随天元(ウズイテンゲン): 年齢:23, 身長:198.0, 強さ:80, 所属:柱, 属性:音
時透無一郎(トキトウムイチロウ): 年齢:14, 身長:160.0, 強さ:90, 所属:柱, 属性:霞
甘露寺蜜璃(カンロジミツリ): 年齢:19, 身長:167.0, 強さ:70, 所属:柱, 属性:恋
伊黒小芭内(イグロオバナイ): 年齢:21, 身長:162.0, 強さ:70, 所属:柱, 属性:蛇
不死川玄弥(シナズガワゲンヤ): 年齢:16, 身長:180.0, 強さ:30, 所属:五感, 属性:鬼
不死川実弥(シナズガワサネミ): 年齢:21, 身長:179.0, 強さ:90, 所属:柱, 属性:風
悲鳴嶋行冥(ヒメジマギョウメイ): 年齢:27, 身長:220.0, 強さ:100, 所属:柱, 属性:岩
initial 2:
鬼舞辻無惨(キブツジムザン): 年齢:1100, 身長:185.0, 強さ:1000, 所属:始祖, 属性:鬼
塁(ルイ): 年齢:150, 身長:148.0, 強さ:50, 所属:下弦, 属性:鬼
轆轤(ロクロ): 年齢:200, 身長:172.5, 強さ:45, 所属:下弦, 属性:鬼
病葉(ワクラバ): 年齢:180, 身長:165.0, 強さ:40, 所属:下弦, 属性:鬼
零余子(

#演習テスト 課題2 バケットソート

メンバの要素(整数)を使ってバケットソートを行うプログラムを完成させよ。

1. 年齢(age)の若い順にバケットソートせよ
2. 強さ(score)の高い順にバケットソートせよ


##要素の整数値(バケットのインデックス)を返す関数

In [72]:
%%file get_index.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"

int get_index_from_age(const Member *member) {
    return member->age;
}

int get_index_from_score(const Member *member) {
    return member->score;
}

int max_index(int num, Member *member[], int (*get_index)(const Member *)) {
    int max = 0;
    for (int i = 0; i < num; i++) {
        int index = get_index(member[i]);
        if (index > max)
            max = index;
    }
    return max+1;
}

Overwriting get_index.c


##バケットソートのプログラム

In [73]:
%%file bucket_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"

// メンバのリストを定義
typedef struct member_list {
    Member *member;
    struct member_list *next;
} MemberList;

// 新しいノードを確保し、member がメンバを指し、next にはNULLを入れる
MemberList *new_member_list(Member *member) {
    MemberList *node;
    node = (MemberList*)malloc(sizeof(MemberList));
    node->member = member;
    node->next = NULL;
    return node;
}

// バケットソート関数
// 入力：max_index: 最大のインデックス(+1)＝バケットのサイズ、num: member_arrayの長さ、member_array: Member*の配列
//       get_index: インデックスを返す関数へのポインタ、descending: 0なら昇順、0以外なら降順
// 出力：新たに確保されたソート済みのMember*の配列を返す
Member **bucket_sort(int max_index, int num, Member *member_array[], int (*get_index)(const Member *), int descending) {
    MemberList *buckets;
    Member **sorted_array;

    reset_counter();
    // バケット配列の確保
    buckets = (MemberList*)malloc(max_index * sizeof(MemberList));
    if (buckets == NULL) {
        printf("Cannot allocate buckets\n");
        return NULL;
    }
    // ソート済みのMember*を格納する配列の確保
    sorted_array = (Member**)malloc(num * sizeof(Member*));
    if (sorted_array == NULL) {
        printf("Cannot allocate sorted_array\n");
        return NULL;
    }
    for (int b = 0; b < max_index; b++) {
        // NULLで初期化
        buckets[b].member = NULL;
        buckets[b].next   = NULL;
    }
    // ここから
    for (int i = 0; i < num; i++) {
        int index = get_index(member_array[i]);
        MemberList *memlist = new_member_list(member_array[i]);

        memlist->next = buckets[index].next;
        buckets[index].next = memlist;
        ctr.move++;
    }

   int p = 0;
   if (!descending) {
     for (int b = 0; b < max_index; b++) {
         for (MemberList *m = buckets[b].next; m != NULL; m = m->next) {
             sorted_array[p++] = m->member;
             ctr.move++;
            }
        }
   } else {
       for (int b = max_index - 1; b >= 0; b--) {
           for (MemberList *m = buckets[b].next; m != NULL; m = m->next) {
               sorted_array[p++] = m->member;
               ctr.move++;
           }
       }
   }

    // ここまでを埋める
    free(member_array);
    return sorted_array;
}

Overwriting bucket_sort.c


###課題2-1 メインプログラム

In [74]:
%%file test2-1.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"

int get_index_from_age(const Member *member);
int get_index_from_score(const Member *member);
int max_index(int num, Member *member[], int (*)(const Member *));

int main(void) {
    Member **member_array, **sorted_array;
    int num_members;
    int max_age;

    // メンバーの読み込み
    member_array = init_members_from_file("members.csv", &num_members);

    printf("initial:\n");
    printarray(num_members, member_array);

    LAUNCH_SORT(num_members, member_array, quick_sort, compare_by_yomi, 0);

    // 年齢(age)でバケットソート
    printf("\nbucket_sort get_age(ascending):\n");
    max_age = max_index(num_members, member_array, get_index_from_age);
    sorted_array = bucket_sort(max_age, num_members, member_array,
                               get_index_from_age, 0);
    printarray(num_members, sorted_array);
    print_counter();

    return 0;
}

Overwriting test2-1.c


###コンパイル(test2-1.c)＆リンク

In [75]:
!gcc -c bucket_sort.c
!gcc -c get_index.c
!gcc -c test2-1.c
!gcc tool.o compare.o bubble_sort.o bubble2_sort.o shaker_sort.o insert_sort.o select_sort.o quick_sort.o heap_sort.o merge_sort.o bucket_sort.o get_index.o test2-1.o -o test2-1

###実行

In [76]:
!./test2-1

initial:
竈門炭治郎(カマドタンジロウ): 年齢:16, 身長:165.0, 強さ:40, 所属:蒲鉾, 属性:水
冨岡義勇(トミオカギユウ): 年齢:21, 身長:176.0, 強さ:80, 所属:柱, 属性:水
竈門禰豆子(カマドネズコ): 年齢:14, 身長:150.0, 強さ:30, 所属:蒲鉾, 属性:鬼
我妻善逸(アガツマゼンイツ): 年齢:16, 身長:164.5, 強さ:40, 所属:蒲鉾, 属性:雷
栗花落カナヲ(ツユリカナヲ): 年齢:16, 身長:156.0, 強さ:50, 所属:五感, 属性:花
嘴平伊之助(ハシビライノスケ): 年齢:15, 身長:164.0, 強さ:40, 所属:蒲鉾, 属性:獣
胡蝶しのぶ(コチョウシノブ): 年齢:18, 身長:151.0, 強さ:70, 所属:柱, 属性:蟲
煉獄杏寿郎(レンゴクキョウジュロウ): 年齢:20, 身長:177.0, 強さ:80, 所属:柱, 属性:炎
宇随天元(ウズイテンゲン): 年齢:23, 身長:198.0, 強さ:80, 所属:柱, 属性:音
時透無一郎(トキトウムイチロウ): 年齢:14, 身長:160.0, 強さ:90, 所属:柱, 属性:霞
甘露寺蜜璃(カンロジミツリ): 年齢:19, 身長:167.0, 強さ:70, 所属:柱, 属性:恋
伊黒小芭内(イグロオバナイ): 年齢:21, 身長:162.0, 強さ:70, 所属:柱, 属性:蛇
不死川玄弥(シナズガワゲンヤ): 年齢:16, 身長:180.0, 強さ:30, 所属:五感, 属性:鬼
不死川実弥(シナズガワサネミ): 年齢:21, 身長:179.0, 強さ:90, 所属:柱, 属性:風
悲鳴嶋行冥(ヒメジマギョウメイ): 年齢:27, 身長:220.0, 強さ:100, 所属:柱, 属性:岩

quick_sort with compare_by_yomi(ascending)

bucket_sort get_age(ascending):
時透無一郎(トキトウムイチロウ): 年齢:14, 身長:160.0, 強さ:90, 所属:柱, 属性:霞
竈門禰豆子(カマドネズコ): 年齢:14, 身長:150.0, 強さ:30, 所属:蒲鉾, 属性:鬼
嘴平伊之助(ハシビライノスケ): 年齢:15, 身長:16

###課題2-2 メインプログラム

In [77]:
%%file test2-2.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"

int get_index_from_age(const Member *member);
int get_index_from_score(const Member *member);
int max_index(int num, Member *member[], int (*)(const Member *));

int main(void) {
    Member **member_array, **sorted_array;
    int num_members;
    int max_score;

    // メンバーの読み込み
    member_array = init_members_from_file("members.csv", &num_members);
    member_array = add_members_from_file("members3.csv", &num_members, member_array);

    printf("initial:\n");
    printarray(num_members, member_array);

    LAUNCH_SORT(num_members, member_array, quick_sort, compare_by_yomi, 0);

    // 強さ(score)の高い順でバケットソート
    printf("\nbucket_sort get_score(descending):\n");
    max_score = max_index(num_members, member_array, get_index_from_score);
    sorted_array = bucket_sort(max_score, num_members, member_array,
                               get_index_from_score, 1);
    printarray(num_members, sorted_array);
    print_counter();

    return 0;
}

Overwriting test2-2.c


###コンパイル(test2-2.c)＆リンク

In [78]:
!gcc -c bucket_sort.c
!gcc -c get_index.c
!gcc -c test2-2.c
!gcc tool.o compare.o bubble_sort.o bubble2_sort.o shaker_sort.o insert_sort.o select_sort.o quick_sort.o heap_sort.o merge_sort.o bucket_sort.o get_index.o test2-2.o -o test2-2

###実行

In [79]:
!./test2-2

initial:
竈門炭治郎(カマドタンジロウ): 年齢:16, 身長:165.0, 強さ:40, 所属:蒲鉾, 属性:水
冨岡義勇(トミオカギユウ): 年齢:21, 身長:176.0, 強さ:80, 所属:柱, 属性:水
竈門禰豆子(カマドネズコ): 年齢:14, 身長:150.0, 強さ:30, 所属:蒲鉾, 属性:鬼
我妻善逸(アガツマゼンイツ): 年齢:16, 身長:164.5, 強さ:40, 所属:蒲鉾, 属性:雷
栗花落カナヲ(ツユリカナヲ): 年齢:16, 身長:156.0, 強さ:50, 所属:五感, 属性:花
嘴平伊之助(ハシビライノスケ): 年齢:15, 身長:164.0, 強さ:40, 所属:蒲鉾, 属性:獣
胡蝶しのぶ(コチョウシノブ): 年齢:18, 身長:151.0, 強さ:70, 所属:柱, 属性:蟲
煉獄杏寿郎(レンゴクキョウジュロウ): 年齢:20, 身長:177.0, 強さ:80, 所属:柱, 属性:炎
宇随天元(ウズイテンゲン): 年齢:23, 身長:198.0, 強さ:80, 所属:柱, 属性:音
時透無一郎(トキトウムイチロウ): 年齢:14, 身長:160.0, 強さ:90, 所属:柱, 属性:霞
甘露寺蜜璃(カンロジミツリ): 年齢:19, 身長:167.0, 強さ:70, 所属:柱, 属性:恋
伊黒小芭内(イグロオバナイ): 年齢:21, 身長:162.0, 強さ:70, 所属:柱, 属性:蛇
不死川玄弥(シナズガワゲンヤ): 年齢:16, 身長:180.0, 強さ:30, 所属:五感, 属性:鬼
不死川実弥(シナズガワサネミ): 年齢:21, 身長:179.0, 強さ:90, 所属:柱, 属性:風
悲鳴嶋行冥(ヒメジマギョウメイ): 年齢:27, 身長:220.0, 強さ:100, 所属:柱, 属性:岩
鬼舞辻無惨(キブツジムザン): 年齢:1100, 身長:185.0, 強さ:1000, 所属:始祖, 属性:鬼
塁(ルイ): 年齢:150, 身長:148.0, 強さ:50, 所属:下弦, 属性:鬼
轆轤(ロクロ): 年齢:200, 身長:172.5, 強さ:45, 所属:下弦, 属性:鬼
病葉(ワクラバ): 年齢:180, 身長:165.0, 強さ:40, 所属:下弦, 属性:鬼
零余子(ムカゴ): 年齢:160,

#発展課題: 3本の配列をマージするマージソート

3つのファイルからそれぞれメンバを読み込み、それぞれをソートした上で、その3本の配列をマージする関数を完成せよ。

##3本の配列をマージするマージソート

In [80]:
%%file merge_three_arrays.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"
Member **merge_three_arrays(int num1, Member *data1[],
                            int num2, Member *data2[],
                            int num3, Member *data3[],
                            CmpFunc cmp, int descending, int *num_merged){
    Member **merged = (Member**)malloc((num1+num2+num3) * sizeof(Member*));
    int i, j, k, p;
    printf("merge_three_arrays.\n");
    reset_counter();
    i = 0; j = 0; k = 0; p = 0;
    // ここから
while (i < num1 || j < num2 || k < num3) {
        Member *m1 = (i < num1) ? data1[i] : NULL;
        Member *m2 = (j < num2) ? data2[j] : NULL;
        Member *m3 = (k < num3) ? data3[k] : NULL;

        Member *min_member = NULL;
        int source = -1;

        if (m1 != NULL) {
            min_member = m1;
            source = 1;
        }
        if (m2 != NULL && (min_member == NULL || cmp(m2, min_member, descending) < 0)) {
            min_member = m2;
            source = 2;
        }
        if (m3 != NULL && (min_member == NULL || cmp(m3, min_member, descending) < 0)) {
            min_member = m3;
            source = 3;
        }

        if (source == 1) i++;
        else if (source == 2) j++;
        else if (source == 3) k++;

        merged[p++] = min_member;
        ctr.move++;
        ctr.compare += 2;  // 2回の比較が最大で起こるため
    }

    // ここまでを埋める
    *num_merged = num1 + num2 + num3;
    free(data1);
    free(data2);
    free(data3);
    return merged;
}

Overwriting merge_three_arrays.c


###メインプログラム

In [81]:
%%file test4.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "main.h"

int main(void) {
    Member **member_array_1, **member_array_2, **member_array_3, **merged_array;
    int num_members_1, num_members_2, num_members_3, num_merged;

    // メンバー1の読み込み
    member_array_1 = init_members_from_file("members.csv", &num_members_1);

    printf("initial 1:\n");
    printarray(num_members_1, member_array_1);

    // メンバー2の読み込み
    member_array_2 = init_members_from_file("members3.csv", &num_members_2);

    printf("initial 2:\n");
    printarray(num_members_2, member_array_2);

    // メンバー3の読み込み
    member_array_3 = init_members_from_file("members4.csv", &num_members_3);

    printf("initial 3:\n");
    printarray(num_members_3, member_array_3);

    // 名前の読み(yomi)順にソート
    LAUNCH_SORT(num_members_1, member_array_1, quick_sort, compare_by_yomi, 0);
    printarray(num_members_1, member_array_1);
    print_counter();

    // 名前の読み(yomi)順にソート
    LAUNCH_SORT(num_members_2, member_array_2, quick_sort, compare_by_yomi, 0);
    printarray(num_members_2, member_array_2);
    print_counter();

    // 名前の読み(yomi)順にソート
    LAUNCH_SORT(num_members_3, member_array_3, quick_sort, compare_by_yomi, 0);
    printarray(num_members_3, member_array_3);
    print_counter();

    // 3つの配列をマージ
    printf("\n");
    merged_array = merge_three_arrays(num_members_1, member_array_1,
                                      num_members_2, member_array_2,
                                      num_members_3, member_array_3,
                                      compare_by_yomi, 0, &num_merged);
    printf("num_merged = %d\n", num_merged);
    printarray(num_merged, merged_array);
    print_counter();

    return 0;
}

Overwriting test4.c


###追加メンバ定義

In [82]:
%%file members4.csv
12
錆兎,サビト,鬼殺,水,13,155.0,45
胡蝶カナエ,コチョウカナエ,柱,花,22,148.5,50
珠世,タマヨ,鬼殺,鬼,600,152.5,30
愈史郎,ユシロウ,鬼殺,鬼,200,148.0,25
鱗滝左近次,ウロコダキサコンジ,育手,水,55,170.5,60
桑島慈悟郎,クワジマジゴロウ,育手,雷,60,165.5,50
煉獄愼寿郎,レンゴクシンジュロウ,柱,炎,45,175.5,60
鋼鐵塚蛍,ハガネヅカホタル,刀鍛冶,刀,26,175.0,10
産屋敷耀哉,ウブヤシキカガヤ,鬼殺,館,35,168.0,10
鉄穴森鋼造,カナモリコウゾウ,刀鍛冶,刀,28,168.0,10
継国縁壱,ツグクニヨリイチ,柱,日,82,182.5,500
鉄地河原鉄珍,テッチカワハラテッチン,刀鍛冶,刀,56,148.0,10

Overwriting members4.csv


###コンパイル（test4.c）＆リンク

In [83]:
!gcc merge_three_arrays.c -c
!gcc test4.c -c
!gcc tool.o compare.o bubble_sort.o bubble2_sort.o shaker_sort.o insert_sort.o select_sort.o quick_sort.o heap_sort.o merge_sort.o merge_two_arrays.o merge_three_arrays.o test4.o -o test4

###実行

In [84]:
!./test4

initial 1:
竈門炭治郎(カマドタンジロウ): 年齢:16, 身長:165.0, 強さ:40, 所属:蒲鉾, 属性:水
冨岡義勇(トミオカギユウ): 年齢:21, 身長:176.0, 強さ:80, 所属:柱, 属性:水
竈門禰豆子(カマドネズコ): 年齢:14, 身長:150.0, 強さ:30, 所属:蒲鉾, 属性:鬼
我妻善逸(アガツマゼンイツ): 年齢:16, 身長:164.5, 強さ:40, 所属:蒲鉾, 属性:雷
栗花落カナヲ(ツユリカナヲ): 年齢:16, 身長:156.0, 強さ:50, 所属:五感, 属性:花
嘴平伊之助(ハシビライノスケ): 年齢:15, 身長:164.0, 強さ:40, 所属:蒲鉾, 属性:獣
胡蝶しのぶ(コチョウシノブ): 年齢:18, 身長:151.0, 強さ:70, 所属:柱, 属性:蟲
煉獄杏寿郎(レンゴクキョウジュロウ): 年齢:20, 身長:177.0, 強さ:80, 所属:柱, 属性:炎
宇随天元(ウズイテンゲン): 年齢:23, 身長:198.0, 強さ:80, 所属:柱, 属性:音
時透無一郎(トキトウムイチロウ): 年齢:14, 身長:160.0, 強さ:90, 所属:柱, 属性:霞
甘露寺蜜璃(カンロジミツリ): 年齢:19, 身長:167.0, 強さ:70, 所属:柱, 属性:恋
伊黒小芭内(イグロオバナイ): 年齢:21, 身長:162.0, 強さ:70, 所属:柱, 属性:蛇
不死川玄弥(シナズガワゲンヤ): 年齢:16, 身長:180.0, 強さ:30, 所属:五感, 属性:鬼
不死川実弥(シナズガワサネミ): 年齢:21, 身長:179.0, 強さ:90, 所属:柱, 属性:風
悲鳴嶋行冥(ヒメジマギョウメイ): 年齢:27, 身長:220.0, 強さ:100, 所属:柱, 属性:岩
initial 2:
鬼舞辻無惨(キブツジムザン): 年齢:1100, 身長:185.0, 強さ:1000, 所属:始祖, 属性:鬼
塁(ルイ): 年齢:150, 身長:148.0, 強さ:50, 所属:下弦, 属性:鬼
轆轤(ロクロ): 年齢:200, 身長:172.5, 強さ:45, 所属:下弦, 属性:鬼
病葉(ワクラバ): 年齢:180, 身長:165.0, 強さ:40, 所属:下弦, 属性:鬼
零余子(